# Lab | Hypothesis Testing

**Objective**

Welcome to the Hypothesis Testing Lab, where we embark on an enlightening journey through the realm of statistical decision-making! In this laboratory, we delve into various scenarios, applying the powerful tools of hypothesis testing to scrutinize and interpret data.

From testing the mean of a single sample (One Sample T-Test), to investigating differences between independent groups (Two Sample T-Test), and exploring relationships within dependent samples (Paired Sample T-Test), our exploration knows no bounds. Furthermore, we'll venture into the realm of Analysis of Variance (ANOVA), unraveling the complexities of comparing means across multiple groups.

So, grab your statistical tools, prepare your hypotheses, and let's embark on this fascinating journey of exploration and discovery in the world of hypothesis testing!

**Challenge 1**

In this challenge, we will be working with pokemon data. The data can be found here:

- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/pokemon.csv

In [1]:
#libraries
import pandas as pd
import scipy.stats as st
import numpy as np


In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/pokemon.csv")
df

,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,Charmander,Fire,NaN,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...
795,Diancie,Rock,Fairy,50,100,150,100,150,50,6,True
796,Mega Diancie,Rock,Fairy,50,160,110,160,110,110,6,True
797,Hoopa Confined,Psychic,Ghost,80,110,60,150,130,70,6,True
798,Hoopa Unbound,Psychic,Dark,80,160,60,170,130,80,6,True


In [8]:
df['Type 2'].unique()

array(['Poison', nan, 'Flying', 'Dragon', 'Ground', 'Fairy', 'Grass',
       'Fighting', 'Psychic', 'Steel', 'Ice', 'Rock', 'Dark', 'Water',
       'Electric', 'Fire', 'Ghost', 'Bug', 'Normal'], dtype=object)

- We posit that Pokemons of type Dragon have, on average, more HP stats than Grass. Choose the propper test and, with 5% significance, comment your findings.

In [40]:
df.dtypes

Name           object
Type 1         object
Type 2         object
HP              int64
Attack          int64
Defense         int64
Sp. Atk         int64
Sp. Def         int64
Speed           int64
Generation      int64
Legendary        bool
total_power     int64
hp_stats        int64
hp_stats_2      int64
dtype: object

In [28]:
df

,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,total_power,hp_stats
0,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False,319,319
1,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False,406,406
2,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False,526,526
3,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False,626,626
4,Charmander,Fire,NaN,39,52,43,60,50,65,1,False,310,310
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,Diancie,Rock,Fairy,50,100,150,100,150,50,6,True,606,606
796,Mega Diancie,Rock,Fairy,50,160,110,160,110,110,6,True,706,706
797,Hoopa Confined,Psychic,Ghost,80,110,60,150,130,70,6,True,606,606
798,Hoopa Unbound,Psychic,Dark,80,160,60,170,130,80,6,True,686,686


In [ ]:
df['hp_stats'] = df['HP'] + df['Attack'] + df['Defense']+df['Sp. Atk']+df['Sp. Def']+df['Speed']+df['Generation']

In [36]:
#code here
# H0 = Dragon have on average the same HP stats as Grass
# H1 = Dragon have on average more HP stats than grass

dragon_df = df[df['Type 1'] == 'Dragon']['HP']
grass_df = df[df['Type 1'] == 'Grass']['HP']
st.ttest_ind(dragon_df,grass_df, equal_var=False, alternative="greater")


TtestResult(statistic=3.3349632905124063, pvalue=0.0007993609745420598, df=50.83784116232685)

The pvalue close to 0 shows that we can reject the null hypothesis and we can accept the alternative hypothesis that Dragons have on average more HP stats than Grass

- We posit that Legendary Pokemons have different stats (HP, Attack, Defense, Sp.Atk, Sp.Def, Speed) when comparing with Non-Legendary. Choose the propper test and, with 5% significance, comment your findings.


In [42]:
# H0 = The two datasets have the same stats
# H1 = The two datasets have different stats

legendary_df = df[df["Legendary"]== True]['hp_stats']
non_legendary_df = df[df['Legendary']==False]['hp_stats']
st.ttest_ind(legendary_df,non_legendary_df, equal_var=False)


TtestResult(statistic=25.94271112418893, pvalue=5.526963641866027e-47, df=103.05878398870199)

Since the p-value is close to 0, we can reject the null hypothesis: The two datasets have different stats

**Challenge 2**

In this challenge, we will be working with california-housing data. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/california_housing.csv

In [43]:
df = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/california_housing.csv")
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


**We posit that houses close to either a school or a hospital are more expensive.**

- School coordinates (-118, 37)
- Hospital coordinates (-122, 34)

We consider a house (neighborhood) to be close to a school or hospital if the distance is lower than 0.50.

Hint:
- Write a function to calculate euclidean distance from each house (neighborhood) to the school and to the hospital.
- Divide your dataset into houses close and far from either a hospital or school.
- Choose the propper test and, with 5% significance, comment your findings.
 

In [ ]:
# H0 = School and hospital proximity are equal or cheaper
# H1 = School and hospital proximity are more expensive

In [ ]:
# for long, lat in df['longitude'], df['latitude']:
#     print(long, lat)
# coordinate = (df['longitude'], df['latitude'])
        # coordinate = (x,y)
# print(coordinate)
    # df['distance_from_school'] = np.linalg.norm(coordinate - school_coordinates)
    # df['distance_from_hospital'] = np.linalg.norm(coordinate - school_coordinates)
# df['close_to_school'] = if df['distance_from_school'] < 5
# df['distance_school'] = np.sqrt
# c = np.sqrt(school x - house x) ** 2) + (school y - house y)**2)
df['longitude']
df['latitude']
school_coordinates = (-118, 37)
hospital_coordinates = (-122, 34)

df['coordinates'] = [(x,y) for x,y in zip(df['longitude'], df['latitude'])]
tuple_to_add = (-118, 37)

In [78]:
df['first_part_school'] = (-118 - df['longitude'])**2

df['second_part_school'] = (37 - df['latitude'])**2
df['distance_from_school'] = np.sqrt(df['first_part_school'] + df['second_part_school'] )

df['first_part_hospital'] = (-122 - df['longitude'])**2

df['second_part_hospital'] = (34 - df['latitude'])**2
df['distance_from_hospital'] = np.sqrt(df['first_part_hospital'] + df['second_part_hospital'] )

In [95]:
df['close'] = (df['distance_from_school'] <= 0.5) | (df['distance_from_hospital'] <= 0.5)

In [96]:
x = df[df["close"]== True]['median_house_value']

y = df[df["close"]== False]['median_house_value']
st.ttest_ind(x,y, equal_var=False, alternative="greater")


TtestResult(statistic=-17.174167998688404, pvalue=0.9999738999071939, df=4.145382282040222)

We failed to reject null hypotesis. Houses closer to school are equal or less expensive.

In [100]:
x.mean(), y.mean()

(92460.0, 207334.6990291262)

In [99]:
y

0         66900.0
1         80100.0
2         85700.0
3         73400.0
4         65500.0
           ...   
16995    111400.0
16996     79000.0
16997    103600.0
16998     85800.0
16999     94600.0
Name: median_house_value, Length: 16995, dtype: float64